## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Jan. 5th Jan. 12th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,January-6-2025,Condo/Co-op,92 SW 3rd St #4411,Miami,FL,33130.0,595000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/92-SW-3rd-St-3...,MARMLS,A11712487,N,Y,25.769593,-80.195047


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
# df2 = df[(df['SOLD DATE'] > '2024-12-29') & (df['SOLD DATE'] < '2025-01-07')]

In [60]:
df2 = df

In [62]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-01-06    28
2025-01-10    24
2025-01-08    15
2025-01-07    14
2025-01-09    12
Name: count, dtype: int64

In [64]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [113]:
df_top_ten = df2.head(10)

In [115]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
35,PAST SALE,2025-01-10,Condo/Co-op,2655 S Bayshore Dr Unit 1702-06,Miami,FL,33133.0,8620875.0,5.0,4.5,Coconut Grove,3636.0,NaN,2024.0,NaN,2371.0,9089.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2655-S-Bayshor...,MARMLS,A11648536,N,Y,25.731196,-80.236270
88,PAST SALE,2025-01-06,Condo/Co-op,350 Ocean Dr Unit 1206N,Key Biscayne,FL,33149.0,5850000.0,3.0,4.5,OCEANA KEY BISCAYNE CONDO,2257.0,NaN,2014.0,NaN,2592.0,6904.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,MARMLS,A11602978,N,Y,25.694534,-80.157613
58,PAST SALE,2025-01-07,Condo/Co-op,19955 NE 38th Ct #1204,Aventura,FL,33180.0,2300000.0,3.0,4.5,TOWERS OF PORTO VITA-SOUT,3320.0,NaN,2000.0,NaN,693.0,2914.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19955-NE-38...,MARMLS,A11635515,N,Y,25.961188,-80.126327
77,PAST SALE,2025-01-10,Condo/Co-op,881 Ocean Dr Unit 6G,Key Biscayne,FL,33149.0,1570000.0,2.0,2.0,CASA DEL MAR CONDO,1875.0,NaN,1971.0,NaN,837.0,1448.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/881-Oce...,MARMLS,A11598188,N,Y,25.685659,-80.157201
69,PAST SALE,2025-01-06,Condo/Co-op,19955 NE 38th Ct #2401,Aventura,FL,33180.0,1550000.0,2.0,3.5,TOWERS OF PORTO VITA-SOUT,2400.0,NaN,2000.0,NaN,646.0,4591.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19955-NE-38...,MARMLS,A11590324,N,Y,25.961188,-80.126327
65,PAST SALE,2025-01-06,Condo/Co-op,301 Altara Ave Unit LPH912,Coral Gables,FL,33146.0,1470000.0,3.0,3.5,MERRICK MANOR CONDO,1539.0,NaN,2019.0,NaN,955.0,2140.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/301-Alt...,MARMLS,A11637836,N,Y,25.733644,-80.261796
62,PAST SALE,2025-01-10,Condo/Co-op,3201 NE 183rd St #804,Aventura,FL,33160.0,1460000.0,2.0,3.5,THE PENINSULA CONDO,2984.0,NaN,2003.0,NaN,489.0,2684.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3201-NE-183...,MARMLS,A11662587,N,Y,25.945719,-80.135657
39,PAST SALE,2025-01-06,Condo/Co-op,1700 S Bayshore Ln Unit 2A,Miami,FL,33133.0,1350000.0,3.0,2.0,PORT BYWATER CONDO,1774.0,NaN,1980.0,NaN,761.0,2395.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1700-S-Bayshor...,MARMLS,A11685578,N,Y,25.738769,-80.220125
12,PAST SALE,2025-01-08,Condo/Co-op,253 NE 2nd St #4408,Miami,FL,33132.0,1300000.0,3.0,3.0,VIZCAYNE SOUTH CONDO,1795.0,NaN,2008.0,NaN,724.0,2339.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/253-NE-2nd-St-...,MARMLS,A11694335,N,Y,25.776385,-80.189087
44,PAST SALE,2025-01-06,Condo/Co-op,151 Crandon Blvd #1031,Key Biscayne,FL,33149.0,1175000.0,2.0,2.0,THE EMERALDBAY @ KEY COLO,1547.0,NaN,1980.0,NaN,760.0,1880.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/151-Cra...,MARMLS,A11630561,N,Y,25.696473,-80.161501


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [116]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [117]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [118]:
response_df = pd.DataFrame(response_list)

In [119]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [120]:
df_top_ten = merged_df

In [121]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-01-10,Condo/Co-op,2655 S Bayshore Dr Unit 1702-06,Miami,FL,33133.0,8620875.0,5.0,4.5,Coconut Grove,3636.0,NaN,2024.0,NaN,2371.0,9089.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2655-S-Bayshor...,MARMLS,A11648536,N,Y,25.731196,-80.236270,https://www.redfin.com/FL/Miami/2655-S-Bayshor...,Terrence Brewer,"Grove Real Estate, Title and Legal Services",None,None,Richard Teed,The Agency Florid LLC,None,None
1,PAST SALE,2025-01-06,Condo/Co-op,350 Ocean Dr Unit 1206N,Key Biscayne,FL,33149.0,5850000.0,3.0,4.5,OCEANA KEY BISCAYNE CONDO,2257.0,NaN,2014.0,NaN,2592.0,6904.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,MARMLS,A11602978,N,Y,25.694534,-80.157613,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,Giulietta Ulloa,BHHS EWM Realty,Jennifer Perez,BHHS EWM Realty,Elena Chacon,BHHS EWM Realty,None,None
2,PAST SALE,2025-01-07,Condo/Co-op,19955 NE 38th Ct #1204,Aventura,FL,33180.0,2300000.0,3.0,4.5,TOWERS OF PORTO VITA-SOUT,3320.0,NaN,2000.0,NaN,693.0,2914.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19955-NE-38...,MARMLS,A11635515,N,Y,25.961188,-80.126327,https://www.redfin.com/FL/Aventura/19955-NE-38...,Rosy Lofer,One Sotheby's International Realty,None,None,Karen Matluck,"Compass Florida, LLC",None,None
3,PAST SALE,2025-01-10,Condo/Co-op,881 Ocean Dr Unit 6G,Key Biscayne,FL,33149.0,1570000.0,2.0,2.0,CASA DEL MAR CONDO,1875.0,NaN,1971.0,NaN,837.0,1448.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/881-Oce...,MARMLS,A11598188,N,Y,25.685659,-80.157201,https://www.redfin.com/FL/Key-Biscayne/881-Oce...,Monica Henao,Miller Eaton LLC,None,None,Monica Henao,Miller Eaton LLC,None,None
4,PAST SALE,2025-01-06,Condo/Co-op,19955 NE 38th Ct #2401,Aventura,FL,33180.0,1550000.0,2.0,3.5,TOWERS OF PORTO VITA-SOUT,2400.0,NaN,2000.0,NaN,646.0,4591.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19955-NE-38...,MARMLS,A11590324,N,Y,25.961188,-80.126327,https://www.redfin.com/FL/Aventura/19955-NE-38...,Karen Matluck,"Compass Florida, LLC",Valentina Gonzalez Villalba,"Compass Florida, LLC",Lana Bell,Douglas Elliman,None,None
5,PAST SALE,2025-01-06,Condo/Co-op,301 Altara Ave Unit LPH912,Coral Gables,FL,33146.0,1470000.0,3.0,3.5,MERRICK MANOR CONDO,1539.0,NaN,2019.0,NaN,955.0,2140.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/301-Alt...,MARMLS,A11637836,N,Y,25.733644,-80.261796,https://www.redfin.com/FL/Coral-Gables/301-Alt...,Casey Loundy,One Sotheby's International Re,None,None,Ludmila Bogatov,Bogatov Realty,None,None
6,PAST SALE,2025-01-10,Condo/Co-op,3201 NE 183rd St #804,Aventura,FL,33160.0,1460000.0,2.0,3.5,THE PENINSULA CONDO,2984.0,NaN,2003.0,NaN,489.0,2684.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3201-NE-183...,MARMLS,A11662587,N,Y,25.945719,-80.135657,https://www.redfin.com/FL/Aventura/3201-NE-183...,Denise Rubin,Coldwell Banker Realty,None,None,Eva Bouhadana,Beachfront Realty Inc,None,None
7,PAST SALE,2025-01-06,Condo/Co-op,1700 S Bayshore Ln Unit 2A,Miami,FL,33133.0,1350000.0,3.0,2.0,PORT BYWATER CONDO,1774.0,NaN,1980.0,NaN,761.0,2395.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1700-S-Bayshor...,MARMLS,A11685578,N,Y,25.738769,-80.220125,https://www.redfin.com/FL/Miami/1700-S-Bayshor...,Jennifer Gross,BHHS EWM Realty,None,None,Jennifer Gross,BHHS EWM Realty,None,None
8,PAST SALE,2025-01-08,Condo/Co-op,253 NE 2nd St #4408,Miami,FL,33132.0,1300000.0,3.0,3.0,VIZCAYNE SOUTH CONDO,1795.0,NaN,2008.0,NaN,724.0,2339.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/253-NE-2nd-St-...,MARMLS,A11694335,N,Y,25.776385,-80.189087,https://www.redfin.com/FL/Mia

## Current Week's Values

In [122]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

93


In [123]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$665,359


In [124]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$503


In [125]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$61,878,375


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [22]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Miami-Dade
Input Previous Week Condo Sales Total (number units sold):
92
Input Previous Week Condo Average Sales Price:
646506
Input Previous Week Condo Average PSF:
511
Input Previous Week Condo Sales total (ex: 198_000_000)
59_500_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [126]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [127]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [128]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [129]:
pd.set_option('display.max_columns',None)

In [130]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-10,Condo/Co-op,2655 S Bayshore Dr Unit 1702-06,Miami,FL,33133.0,8620875.0,5.0,4.5,Coconut Grove,3636.0,NaN,2024.0,NaN,2371.0,9089.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2655-S-Bayshor...,MARMLS,A11648536,N,Y,25.731196,-80.23627,https://www.redfin.com/FL/Miami/2655-S-Bayshor...,Terrence Brewer,"Grove Real Estate, Title and Legal Services",None,None,Richard Teed,The Agency Florid LLC,None,None,1,orange


In [131]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [132]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-10,Condo/Co-op,2655 S Bayshore Dr Unit 1702-06,Miami,FL,33133.0,8620875.0,5.0,4.5,Coconut Grove,3636.0,NaN,2024.0,NaN,2371.0,9089.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2655-S-Bayshor...,MARMLS,A11648536,N,Y,25.731196,-80.236270,https://www.redfin.com/FL/Miami/2655-S-Bayshor...,Terrence Brewer,"Grove Real Estate, Title and Legal Services",None,None,Richard Teed,The Agency Florid LLC,None,None,1,orange
1,PAST SALE,2025-01-06,Condo/Co-op,350 Ocean Dr Unit 1206N,Key Biscayne,FL,33149.0,5850000.0,3.0,4.5,OCEANA KEY BISCAYNE CONDO,2257.0,NaN,2014.0,NaN,2592.0,6904.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,MARMLS,A11602978,N,Y,25.694534,-80.157613,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,Giulietta Ulloa,BHHS EWM Realty,Jennifer Perez,BHHS EWM Realty,Elena Chacon,BHHS EWM Realty,None,None,2,blue
2,PAST SALE,2025-01-07,Condo/Co-op,19955 NE 38th Ct #1204,Aventura,FL,33180.0,2300000.0,3.0,4.5,TOWERS OF PORTO VITA-SOUT,3320.0,NaN,2000.0,NaN,693.0,2914.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19955-NE-38...,MARMLS,A11635515,N,Y,25.961188,-80.126327,https://www.redfin.com/FL/Aventura/19955-NE-38...,Rosy Lofer,One Sotheby's International Realty,None,None,Karen Matluck,"Compass Florida, LLC",None,None,3,blue
3,PAST SALE,2025-01-10,Condo/Co-op,881 Ocean Dr Unit 6G,Key Biscayne,FL,33149.0,1570000.0,2.0,2.0,CASA DEL MAR CONDO,1875.0,NaN,1971.0,NaN,837.0,1448.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/881-Oce...,MARMLS,A11598188,N,Y,25.685659,-80.157201,https://www.redfin.com/FL/Key-Biscayne/881-Oce...,Monica Henao,Miller Eaton LLC,None,None,Monica Henao,Miller Eaton LLC,None,None,4,blue
4,PAST SALE,2025-01-06,Condo/Co-op,19955 NE 38th Ct #2401,Aventura,FL,33180.0,1550000.0,2.0,3.5,TOWERS OF PORTO VITA-SOUT,2400.0,NaN,2000.0,NaN,646.0,4591.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19955-NE-38...,MARMLS,A11590324,N,Y,25.961188,-80.126327,https://www.redfin.com/FL/Aventura/19955-NE-38...,Karen Matluck,"Compass Florida, LLC",Valentina Gonzalez Villalba,"Compass Florida, LLC",Lana Bell,Douglas Elliman,None,None,5,blue
5,PAST SALE,2025-01-06,Condo/Co-op,301 Altara Ave Unit LPH912,Coral Gables,FL,33146.0,1470000.0,3.0,3.5,MERRICK MANOR CONDO,1539.0,NaN,2019.0,NaN,955.0,2140.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/301-Alt...,MARMLS,A11637836,N,Y,25.733644,-80.261796,https://www.redfin.com/FL/Coral-Gables/301-Alt...,Casey Loundy,One Sotheby's International Re,None,None,Ludmila Bogatov,Bogatov Realty,None,None,6,blue
6,PAST SALE,2025-01-10,Condo/Co-op,3201 NE 183rd St #804,Aventura,FL,33160.0,1460000.0,2.0,3.5,THE PENINSULA CONDO,2984.0,NaN,2003.0,NaN,489.0,2684.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3201-NE-183...,MARMLS,A11662587,N,Y,25.945719,-80.135657,https://www.redfin.com/FL/Aventura/3201-NE-183...,Denise Rubin,Coldwell Banker Realty,None,None,Eva Bouhadana,Beachfront Realty Inc,None,None,7,blue
7,PAST SALE,2025-01-06,Condo/Co-op,1700 S Bayshore Ln Unit 2A,Miami,FL,33133.0,1350000.0,3.0,2.0,PORT BYWATER CONDO,1774.0,NaN,1980.0,NaN,761.0,2395.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1700-S-Bayshor...,MARMLS,A11685578,N,Y,25.738769,-80.220125,https://www.redfin.com/FL/Miami/1700-S-Bayshor...,Jennifer Gross,BHHS EWM Realty,None,None,Jennifer Gross,BHHS EWM Realty,None,None,8,blue
8,PAST SALE,2025-01-08,Condo/Co-op,253 NE 2nd St #4408,Miami,FL,33132.0,1300000.0,3.0,3.0,VIZCAYNE SOUTH CONDO,1795.0,NaN,2008.0,NaN,724.0,2339.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/253-NE-2nd-St-...,MA

In [133]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [134]:
m.save('index.html')

## Data snagger

In [135]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [136]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-01-10,Condo/Co-op,2655 S Bayshore Dr Unit 1702-06,Miami,FL,33133.0,8620875.0,5.0,4.5,Coconut Grove,3636.0,NaN,2024.0,NaN,2371.0,9089.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2655-S-Bayshor...,MARMLS,A11648536,N,Y,25.731196,-80.23627,https://www.redfin.com/FL/Miami/2655-S-Bayshor...,Terrence Brewer,"Grove Real Estate, Title and Legal Services",None,None,Richard Teed,The Agency Florid LLC,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [137]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiDade_County_condo_sales_week_ending_01132024


# CREATE TEMPLATE 

In [138]:
muni_set = set(df_top_ten['CITY'])

In [139]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [140]:
df_top_ten.reset_index(inplace=True,drop=True)

In [141]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [142]:
top_sale

'Coconut Grove at 2655 S Bayshore Dr Unit 1702-06 in Miami'

In [143]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [144]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [145]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-10,Condo/Co-op,2655 S Bayshore Dr Unit 1702-06,Miami,FL,33133.0,8620875.0,5.0,4.5,Coconut Grove,3636.0,NaN,2024.0,NaN,2371.0,9089.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2655-S-Bayshor...,MARMLS,A11648536,N,Y,25.731196,-80.236270,https://www.redfin.com/FL/Miami/2655-S-Bayshor...,Terrence Brewer,"Grove Real Estate, Title and Legal Services",None,None,Richard Teed,The Agency Florid LLC,None,None,1,orange,Coconut Grove at 2655 S Bayshore Dr Unit 1702-...
1,PAST SALE,2025-01-06,Condo/Co-op,350 Ocean Dr Unit 1206N,Key Biscayne,FL,33149.0,5850000.0,3.0,4.5,OCEANA KEY BISCAYNE CONDO,2257.0,NaN,2014.0,NaN,2592.0,6904.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,MARMLS,A11602978,N,Y,25.694534,-80.157613,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,Giulietta Ulloa,BHHS EWM Realty,Jennifer Perez,BHHS EWM Realty,Elena Chacon,BHHS EWM Realty,None,None,2,blue,OCEANA KEY BISCAYNE CONDO at 350 Ocean Dr Unit...
2,PAST SALE,2025-01-07,Condo/Co-op,19955 NE 38th Ct #1204,Aventura,FL,33180.0,2300000.0,3.0,4.5,TOWERS OF PORTO VITA-SOUT,3320.0,NaN,2000.0,NaN,693.0,2914.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19955-NE-38...,MARMLS,A11635515,N,Y,25.961188,-80.126327,https://www.redfin.com/FL/Aventura/19955-NE-38...,Rosy Lofer,One Sotheby's International Realty,None,None,Karen Matluck,"Compass Florida, LLC",None,None,3,blue,TOWERS OF PORTO VITA-SOUT at 19955 NE 38th Ct ...
3,PAST SALE,2025-01-10,Condo/Co-op,881 Ocean Dr Unit 6G,Key Biscayne,FL,33149.0,1570000.0,2.0,2.0,CASA DEL MAR CONDO,1875.0,NaN,1971.0,NaN,837.0,1448.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/881-Oce...,MARMLS,A11598188,N,Y,25.685659,-80.157201,https://www.redfin.com/FL/Key-Biscayne/881-Oce...,Monica Henao,Miller Eaton LLC,None,None,Monica Henao,Miller Eaton LLC,None,None,4,blue,CASA DEL MAR CONDO at 881 Ocean Dr Unit 6G in ...
4,PAST SALE,2025-01-06,Condo/Co-op,19955 NE 38th Ct #2401,Aventura,FL,33180.0,1550000.0,2.0,3.5,TOWERS OF PORTO VITA-SOUT,2400.0,NaN,2000.0,NaN,646.0,4591.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19955-NE-38...,MARMLS,A11590324,N,Y,25.961188,-80.126327,https://www.redfin.com/FL/Aventura/19955-NE-38...,Karen Matluck,"Compass Florida, LLC",Valentina Gonzalez Villalba,"Compass Florida, LLC",Lana Bell,Douglas Elliman,None,None,5,blue,TOWERS OF PORTO VITA-SOUT at 19955 NE 38th Ct ...
5,PAST SALE,2025-01-06,Condo/Co-op,301 Altara Ave Unit LPH912,Coral Gables,FL,33146.0,1470000.0,3.0,3.5,MERRICK MANOR CONDO,1539.0,NaN,2019.0,NaN,955.0,2140.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/301-Alt...,MARMLS,A11637836,N,Y,25.733644,-80.261796,https://www.redfin.com/FL/Coral-Gables/301-Alt...,Casey Loundy,One Sotheby's International Re,None,None,Ludmila Bogatov,Bogatov Realty,None,None,6,blue,MERRICK MANOR CONDO at 301 Altara Ave Unit LPH...
6,PAST SALE,2025-01-10,Condo/Co-op,3201 NE 183rd St #804,Aventura,FL,33160.0,1460000.0,2.0,3.5,THE PENINSULA CONDO,2984.0,NaN,2003.0,NaN,489.0,2684.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3201-NE-183...,MARMLS,A11662587,N,Y,25.945719,-80.135657,https://www.redfin.com/FL/Aventura/3201-NE-183...,Denise Rubin,Coldwell Banker Realty,None,None,Eva Bouhadana,Beachfront Realty Inc,None,None,7,blue,THE PENINSULA CONDO at 3201 NE 183rd St #804 i...
7,PAST SALE,2025-01-06,Condo/Co-op,1700 S Bayshore Ln Unit 2A,Miami,FL,33133.0,1350000.0,3.0,2.0,PORT BYWATER CONDO,1774.0,NaN,1980.0,NaN,761.0,2395.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1700-S-Bayshor...,MARMLS,A11685578,N,Y,25.738769,-

In [146]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: Coconut Grove closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,175,000 to $8,620,875
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Key Biscayne, Aventura, Coral Gables, Miami

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 93 condo sales totaling $61,878,375 million from Jan. 5th to Jan. 12th. The previous week, brokers closed 92 condo sales totaling $59,500,000.

Last week’s units sold for an average of $665,359, higher than the $646,506 average price from the previous week. The average price per s

In [147]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [148]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [149]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [150]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-10,Condo/Co-op,2655 S Bayshore Dr Unit 1702-06,Miami,FL,33133.0,8620875.0,5.0,4.5,Coconut Grove,3636.0,NaN,2024.0,NaN,2371.0,9089.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2655-S-Bayshor...,MARMLS,A11648536,N,Y,25.731196,-80.236270,https://www.redfin.com/FL/Miami/2655-S-Bayshor...,Terrence Brewer,"Grove Real Estate, Title and Legal Services",None,None,Richard Teed,The Agency Florid LLC,None,None,1,orange,Coconut Grove at 2655 S Bayshore Dr Unit 1702-...
1,PAST SALE,2025-01-06,Condo/Co-op,350 Ocean Dr Unit 1206N,Key Biscayne,FL,33149.0,5850000.0,3.0,4.5,OCEANA KEY BISCAYNE CONDO,2257.0,NaN,2014.0,NaN,2592.0,6904.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,MARMLS,A11602978,N,Y,25.694534,-80.157613,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,Giulietta Ulloa,BHHS EWM Realty,Jennifer Perez,BHHS EWM Realty,Elena Chacon,BHHS EWM Realty,None,None,2,blue,OCEANA KEY BISCAYNE CONDO at 350 Ocean Dr Unit...
2,PAST SALE,2025-01-07,Condo/Co-op,19955 NE 38th Ct #1204,Aventura,FL,33180.0,2300000.0,3.0,4.5,TOWERS OF PORTO VITA-SOUT,3320.0,NaN,2000.0,NaN,693.0,2914.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19955-NE-38...,MARMLS,A11635515,N,Y,25.961188,-80.126327,https://www.redfin.com/FL/Aventura/19955-NE-38...,Rosy Lofer,One Sotheby's International Realty,None,None,Karen Matluck,"Compass Florida, LLC",None,None,3,blue,TOWERS OF PORTO VITA-SOUT at 19955 NE 38th Ct ...
3,PAST SALE,2025-01-10,Condo/Co-op,881 Ocean Dr Unit 6G,Key Biscayne,FL,33149.0,1570000.0,2.0,2.0,CASA DEL MAR CONDO,1875.0,NaN,1971.0,NaN,837.0,1448.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/881-Oce...,MARMLS,A11598188,N,Y,25.685659,-80.157201,https://www.redfin.com/FL/Key-Biscayne/881-Oce...,Monica Henao,Miller Eaton LLC,None,None,Monica Henao,Miller Eaton LLC,None,None,4,blue,CASA DEL MAR CONDO at 881 Ocean Dr Unit 6G in ...
4,PAST SALE,2025-01-06,Condo/Co-op,19955 NE 38th Ct #2401,Aventura,FL,33180.0,1550000.0,2.0,3.5,TOWERS OF PORTO VITA-SOUT,2400.0,NaN,2000.0,NaN,646.0,4591.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19955-NE-38...,MARMLS,A11590324,N,Y,25.961188,-80.126327,https://www.redfin.com/FL/Aventura/19955-NE-38...,Karen Matluck,"Compass Florida, LLC",Valentina Gonzalez Villalba,"Compass Florida, LLC",Lana Bell,Douglas Elliman,None,None,5,blue,TOWERS OF PORTO VITA-SOUT at 19955 NE 38th Ct ...
5,PAST SALE,2025-01-06,Condo/Co-op,301 Altara Ave Unit LPH912,Coral Gables,FL,33146.0,1470000.0,3.0,3.5,MERRICK MANOR CONDO,1539.0,NaN,2019.0,NaN,955.0,2140.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/301-Alt...,MARMLS,A11637836,N,Y,25.733644,-80.261796,https://www.redfin.com/FL/Coral-Gables/301-Alt...,Casey Loundy,One Sotheby's International Re,None,None,Ludmila Bogatov,Bogatov Realty,None,None,6,blue,MERRICK MANOR CONDO at 301 Altara Ave Unit LPH...
6,PAST SALE,2025-01-10,Condo/Co-op,3201 NE 183rd St #804,Aventura,FL,33160.0,1460000.0,2.0,3.5,THE PENINSULA CONDO,2984.0,NaN,2003.0,NaN,489.0,2684.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3201-NE-183...,MARMLS,A11662587,N,Y,25.945719,-80.135657,https://www.redfin.com/FL/Aventura/3201-NE-183...,Denise Rubin,Coldwell Banker Realty,None,None,Eva Bouhadana,Beachfront Realty Inc,None,None,7,blue,THE PENINSULA CONDO at 3201 NE 183rd St #804 i...
7,PAST SALE,2025-01-06,Condo/Co-op,1700 S Bayshore Ln Unit 2A,Miami,FL,33133.0,1350000.0,3.0,2.0,PORT BYWATER CONDO,1774.0,NaN,1980.0,NaN,761.0,2395.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1700-S-Bayshor...,MARMLS,A11685578,N,Y,25.738769,-

In [151]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: Coconut Grove closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,175,000 to $8,620,875
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Miami-Dade condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade, weekly condo sales, Key Biscayne, Aventura, Coral Gables, Miami

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 93 condo sales totaling $61,878,375 million from Jan. 5th to Jan. 12th. The previous week, brokers closed 92 condo sales totaling $59,500,000.

Last week’s units sold for an average of $665,359, higher than the $646,506 average price from the previous week. The average price per s

In [152]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Miami/2655-S-Bayshore-Dr-33133/unit-1702-06/home/179307133


In [160]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Key-Biscayne/350-Ocean-Dr-33149/unit-1206N/home/79138168


In [154]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Key-Biscayne/151-Crandon-Blvd-33149/unit-1031/home/42924909


In [155]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami/253-NE-2nd-St-33132/unit-4408/home/43408491


In [156]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Key-Biscayne/350-Ocean-Dr-33149/unit-1206N/home/79138168


In [157]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Aventura/3201-NE-183rd-St-33160/unit-804/home/43313687


In [158]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-01-10,Condo/Co-op,2655 S Bayshore Dr Unit 1702-06,Miami,FL,33133.0,8620875.0,5.0,4.5,Coconut Grove,3636.0,NaN,2024.0,NaN,2371.0,9089.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2655-S-Bayshor...,MARMLS,A11648536,N,Y,25.731196,-80.236270,https://www.redfin.com/FL/Miami/2655-S-Bayshor...,Terrence Brewer,"Grove Real Estate, Title and Legal Services",None,None,Richard Teed,The Agency Florid LLC,None,None,1,orange,Coconut Grove at 2655 S Bayshore Dr Unit 1702-...
1,PAST SALE,2025-01-06,Condo/Co-op,350 Ocean Dr Unit 1206N,Key Biscayne,FL,33149.0,5850000.0,3.0,4.5,OCEANA KEY BISCAYNE CONDO,2257.0,NaN,2014.0,NaN,2592.0,6904.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,MARMLS,A11602978,N,Y,25.694534,-80.157613,https://www.redfin.com/FL/Key-Biscayne/350-Oce...,Giulietta Ulloa,BHHS EWM Realty,Jennifer Perez,BHHS EWM Realty,Elena Chacon,BHHS EWM Realty,None,None,2,blue,OCEANA KEY BISCAYNE CONDO at 350 Ocean Dr Unit...
2,PAST SALE,2025-01-07,Condo/Co-op,19955 NE 38th Ct #1204,Aventura,FL,33180.0,2300000.0,3.0,4.5,TOWERS OF PORTO VITA-SOUT,3320.0,NaN,2000.0,NaN,693.0,2914.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19955-NE-38...,MARMLS,A11635515,N,Y,25.961188,-80.126327,https://www.redfin.com/FL/Aventura/19955-NE-38...,Rosy Lofer,One Sotheby's International Realty,None,None,Karen Matluck,"Compass Florida, LLC",None,None,3,blue,TOWERS OF PORTO VITA-SOUT at 19955 NE 38th Ct ...
3,PAST SALE,2025-01-10,Condo/Co-op,881 Ocean Dr Unit 6G,Key Biscayne,FL,33149.0,1570000.0,2.0,2.0,CASA DEL MAR CONDO,1875.0,NaN,1971.0,NaN,837.0,1448.0,Sold,NaN,NaN,https://www.redfin.com/FL/Key-Biscayne/881-Oce...,MARMLS,A11598188,N,Y,25.685659,-80.157201,https://www.redfin.com/FL/Key-Biscayne/881-Oce...,Monica Henao,Miller Eaton LLC,None,None,Monica Henao,Miller Eaton LLC,None,None,4,blue,CASA DEL MAR CONDO at 881 Ocean Dr Unit 6G in ...
4,PAST SALE,2025-01-06,Condo/Co-op,19955 NE 38th Ct #2401,Aventura,FL,33180.0,1550000.0,2.0,3.5,TOWERS OF PORTO VITA-SOUT,2400.0,NaN,2000.0,NaN,646.0,4591.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/19955-NE-38...,MARMLS,A11590324,N,Y,25.961188,-80.126327,https://www.redfin.com/FL/Aventura/19955-NE-38...,Karen Matluck,"Compass Florida, LLC",Valentina Gonzalez Villalba,"Compass Florida, LLC",Lana Bell,Douglas Elliman,None,None,5,blue,TOWERS OF PORTO VITA-SOUT at 19955 NE 38th Ct ...
5,PAST SALE,2025-01-06,Condo/Co-op,301 Altara Ave Unit LPH912,Coral Gables,FL,33146.0,1470000.0,3.0,3.5,MERRICK MANOR CONDO,1539.0,NaN,2019.0,NaN,955.0,2140.0,Sold,NaN,NaN,https://www.redfin.com/FL/Coral-Gables/301-Alt...,MARMLS,A11637836,N,Y,25.733644,-80.261796,https://www.redfin.com/FL/Coral-Gables/301-Alt...,Casey Loundy,One Sotheby's International Re,None,None,Ludmila Bogatov,Bogatov Realty,None,None,6,blue,MERRICK MANOR CONDO at 301 Altara Ave Unit LPH...
6,PAST SALE,2025-01-10,Condo/Co-op,3201 NE 183rd St #804,Aventura,FL,33160.0,1460000.0,2.0,3.5,THE PENINSULA CONDO,2984.0,NaN,2003.0,NaN,489.0,2684.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3201-NE-183...,MARMLS,A11662587,N,Y,25.945719,-80.135657,https://www.redfin.com/FL/Aventura/3201-NE-183...,Denise Rubin,Coldwell Banker Realty,None,None,Eva Bouhadana,Beachfront Realty Inc,None,None,7,blue,THE PENINSULA CONDO at 3201 NE 183rd St #804 i...
7,PAST SALE,2025-01-06,Condo/Co-op,1700 S Bayshore Ln Unit 2A,Miami,FL,33133.0,1350000.0,3.0,2.0,PORT BYWATER CONDO,1774.0,NaN,1980.0,NaN,761.0,2395.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1700-S-Bayshor...,MARMLS,A11685578,N,Y,25.738769,-

## Time on Market Calculator

In [161]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 6, 7) ## List (Earlier) date
date2 = datetime(2025, 1, 6) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

213


## Clean CSV for Datawrapper Chart

In [162]:
chart_df = df_top_ten

In [163]:
chart_df = chart_df.fillna(" ")

In [164]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [165]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [166]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [167]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [168]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x))
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [169]:
csv_date_string = today.strftime("%m_%d_%Y")

In [170]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv")

In [171]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-01-10,2655 S Bayshore Dr Unit 1702-06 in Miami,"$8,620,875",5,4.5,"3,636",2024,"$2,371",MARMLS,A11648536,"Terrence Brewer Grove Real Estate, Title and L...",Richard Teed The Agency Florid LLC
1,2025-01-06,350 Ocean Dr Unit 1206N in Key Biscayne,"$5,850,000",3,4.5,"2,257",2014,"$2,592",MARMLS,A11602978,Giulietta Ulloa BHHS EWM Realty Jennifer Perez...,Elena Chacon BHHS EWM Realty
2,2025-01-07,19955 NE 38th Ct #1204 in Aventura,"$2,300,000",3,4.5,"3,320",2000,$693,MARMLS,A11635515,Rosy Lofer One Sotheby's International Realty,"Karen Matluck Compass Florida, LLC"
3,2025-01-10,881 Ocean Dr Unit 6G in Key Biscayne,"$1,570,000",2,2.0,"1,875",1971,$837,MARMLS,A11598188,Monica Henao Miller Eaton LLC,Monica Henao Miller Eaton LLC
4,2025-01-06,19955 NE 38th Ct #2401 in Aventura,"$1,550,000",2,3.5,"2,400",2000,$646,MARMLS,A11590324,"Karen Matluck Compass Florida, LLC Valentina G...",Lana Bell Douglas Elliman
5,2025-01-06,301 Altara Ave Unit LPH912 in Coral Gables,"$1,470,000",3,3.5,"1,539",2019,$955,MARMLS,A11637836,Casey Loundy One Sotheby's International Re,Ludmila Bogatov Bogatov Realty
6,2025-01-10,3201 NE 183rd St #804 in Aventura,"$1,460,000",2,3.5,"2,984",2003,$489,MARMLS,A11662587,Denise Rubin Coldwell Banker Realty,Eva Bouhadana Beachfront Realty Inc
7,2025-01-06,1700 S Bayshore Ln Unit 2A in Miami,"$1,350,000",3,2.0,"1,774",1980,$761,MARMLS,A11685578,Jennifer Gross BHHS EWM Realty,Jennifer Gross BHHS EWM Realty
8,2025-01-08,253 NE 2nd St #4408 in Miami,"$1,300,000",3,3.0,"1,795",2008,$724,MARMLS,A11694335,"Alexandra White Compass Florida, LLC.","Irene Restrepo RES Real Estate Solutions, LLC"
9,2025-01-06,151 Crandon Blvd #1031 in Key Biscayne,"$1,175,000",2,2.0,"1,547",1980,$760,MARMLS,A11630561,"James Salas E Realty International, Inc.","James Salas E Realty International, Inc."
